In [1]:
!python3 -m pip install -r requirements_setup_lora_dataset.txt

  Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached xxhash-3.4.1-cp310-cp310-macosx_11_0_arm64.whl (30 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.4/387.4 kB 2.4 MB/s eta 0:00:000:00:01m eta 0:00:01
  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl (169 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 10.9 MB/s eta 0:00:000:00:010:00:011
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
  Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.

In [7]:
# Imports 
import os 
from datasets import load_dataset, Dataset
from datasets import DatasetDict

In [2]:

directory_path = "lora_dataset"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Created directory: {directory_path}")
else:
    print(f"Directory already exists: {directory_path}")

Created directory: lora_dataset


In [5]:
def prepare_train_data(data):
    data_df = data.to_pandas()
    data_df["text"] = data_df[["description", "color"]].apply(lambda x: "<|im_start|>user\n" + x["description"] + " <|im_end|>\n<|im_start|>assistant\n" + x["color"] + "<|im_end|>\n", axis=1)
    return Dataset.from_pandas(data_df)


In [8]:


# Load your dataset
dataset = load_dataset('burkelibbey/colors', split='train')

# Split the dataset into training (70%), validation (15%), and test (15%)
train_testvalid = dataset.train_test_split(test_size=0.3)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
splits = DatasetDict({
    'train': train_testvalid['train'],
    'valid': test_valid['test'],
    'test': test_valid['train']
})

# Save the splits as JSONL files
for split, split_dataset in splits.items():
    split_dataset = prepare_train_data(split_dataset)
    split_dataset.to_json(f'./lora_dataset/{split}.jsonl', orient='records', lines=True)


Creating json from Arrow format: 100%|██████████| 6/6 [00:00<00:00, 521.38ba/s]
